### Deploy registered AG model in MLFlow for RT Inference

In [2]:
import boto3

import mlflow
import mlflow.sagemaker as mfs

In [4]:
bucket = 'ag-example-timeseries'

# Create S3 client
s3 = boto3.client("s3")


TRACKING_URI = "arn:aws:sagemaker:us-east-1:543531862107:mlflow-tracking-server/ag-ex-timeseries"  # with sagemaker-mlflow plugin
EXPERIMENT   = "autogluon-timeseries"
REGION       = "us-east-1"
EXEC_ROLE_ARN= "arn:aws:iam::<acct>:role/<sagemaker-exec-role>"
APP_NAME     = "ag-ts-endpoint"

### Inference 

In [ ]:
mlflow.set_tracking_uri(TRACKING_URI)
client = mlflow.MlflowClient()

# pick the latest “ready” run (we set this tag in train.py)
runs = client.search_runs(
    [client.get_experiment_by_name(EXPERIMENT).experiment_id],
    filter_string="tags.sagemaker_model_ready = 'true'",
    order_by=["attributes.start_time DESC"],
    max_results=1,
)
if not runs:
    raise RuntimeError("No MLflow runs tagged sagemaker_model_ready=true were found.")
run_id = runs[0].info.run_id
model_uri = f"runs:/{run_id}/model"

mfs.deploy(
    app_name=APP_NAME,
    model_uri=model_uri,
    mode="create",               # "replace" to update in-place
    region_name=REGION,
    execution_role_arn=EXEC_ROLE_ARN,
    instance_type="ml.m5.large",
    instance_count=1,
)
print("Deployed endpoint:", APP_NAME)
